# Causal Supply Chain Analysis Walkthrough

This notebook demonstrates the full workflow of the Causal Supply Chain Toolkit:
1. **Data Ingestion**: Loading synthetic supply chain data.
2. **Causal Discovery**: Learning the causal graph structure.
3. **Effect Estimation**: Estimating the causal effect of interventions.
4. **Simulation**: Simulating "what-if" scenarios.

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add src to path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from utils.data_loader import generate_synthetic_data
from causal.discovery import CausalDiscovery
from causal.estimation import EffectEstimator
from simulation.simulator import SupplyChainSimulator

## 1. Data Ingestion

In [ ]:
# Generate synthetic data
data = generate_synthetic_data(n_samples=5000)
print("Data Shape:", data.shape)
data.head()

## 2. Causal Discovery
We use a heuristic method to discover the causal graph. In a real scenario, you might use domain knowledge or algorithms like PC.

In [ ]:
discovery = CausalDiscovery(data)
graph = discovery.learn_graph(threshold=0.1)
discovery.plot_graph()

## 3. Effect Estimation
We estimate the effect of **Route Choice** (Treatment) on **Delivery Delay** (Outcome), controlling for confounders like **Port Congestion**.

In [ ]:
estimator = EffectEstimator(data)

# Define treatment and outcome
treatment = 'route_choice'
outcome = 'delivery_delay'
common_causes = ['port_congestion', 'weather_shock']

estimate = estimator.estimate_effect(treatment, outcome, common_causes)

print(f"Estimated Effect of {treatment} on {outcome}: {estimate.value:.4f}")

## 4. Simulation (What-If Analysis)
Now we simulate a scenario: What if we forced **all** shipments to use the alternative route?

In [ ]:
# Initialize simulator with the estimated effect
effects = {treatment: estimate.value}
simulator = SupplyChainSimulator(effects)

# Define scenarios
scenarios = [
    {'name': 'Baseline (Current Mix)', 'col': None, 'val': None},
    {'name': 'All Standard Route (0)', 'col': 'route_choice', 'val': 0},
    {'name': 'All Alternative Route (1)', 'col': 'route_choice', 'val': 1}
]

# Compare
results = simulator.compare_scenarios(data, scenarios)
results

In [ ]:
# Visualize Results
results.plot(kind='bar', x='Scenario', y='Avg Delivery Delay', color=['gray', 'red', 'green'])
plt.title("Impact of Route Strategy on Delivery Delay")
plt.ylabel("Average Delay (Days)")
plt.show()